# 📘 StudyMate – Your AI Study Assistant

**StudyMate** is an AI-powered study assistant built to make learning easier, faster, and more personalized. Whether you're preparing for exams, reviewing class materials, or exploring a tough concept, StudyMate acts like a smart tutor in your pocket. It explains topics in simple terms, summarizes long readings, and even quizzes you — all in a friendly, interactive way tailored to your level. Perfect for high school, college, or self-learners who want to study smarter, not harder.

In [ ]:
#install necessary dependency
!pip install PyPDF2

In [ ]:
# imports

import os
from dotenv import load_dotenv
from google import genai
from google.genai import types
import PyPDF2
from openai import OpenAI
import gradio as gr

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
gemini_api_key = os.getenv('GEMINI_API_KEY')

if gemini_api_key:
    print(f"Gemini API Key exists and begins {gemini_api_key[:8]}")
else:
    print("Gemini API Key not set")
    


In [ ]:
system_message= """You are a highly intelligent, helpful, and friendly AI Study Assistant named StudyMate.

Your primary goal is to help students deeply understand academic topics, especially from textbooks, lecture notes, or PDF materials. You must explain concepts clearly, simplify complex ideas, and adapt your responses to the user's grade level and learning style.

Always follow these rules:

1. Break down complex concepts into **simple, digestible explanations** using analogies or examples.
2. If the user asks for a **summary**, provide a concise yet accurate overview of the content.
3. If asked for a **quiz**, generate 3–5 high-quality multiple-choice or short-answer questions.
4. If the user uploads or references a **textbook**, **PDF**, or **paragraph**, use only that context and avoid adding unrelated info.
5. Be interactive. If a user seems confused or asks for clarification, ask helpful guiding questions.
6. Use friendly and motivational tone, but stay focused and to-the-point.
7. Include definitions, bullet points, tables, or emojis when helpful, but avoid unnecessary fluff.
8. If you don't know the answer confidently, say so and recommend a way to find it.

Example roles you may play:
- Explain like a teacher 👩‍🏫
- Summarize like a scholar 📚
- Quiz like an examiner 🧠
- Motivate like a friend 💪

Always ask, at the end:  
*"Would you like me to quiz you, explain another part, or give study tips on this?"*
"""

In [ ]:
# Initialize

class StudyAssistant:
    def __init__(self,api_key):
        gemini= genai.Client(
           api_key= gemini_api_key
        )
        self.gemini = gemini.chats.create(
             model="gemini-2.5-flash",
            config= types.GenerateContentConfig(
                system_instruction= system_message,
                temperature = 0.7
            )
        )

        self.ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
        self.models = {"llma":"llama3:8b","qwen":"qwen2.5:latest"}

    def pdf_extractor(self,pdf_path):
         """Extract text from PDF file"""
         try:
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
                return text.strip()
         except Exception as e:
            return f"Error reading PDF: {str(e)}"

    def chat(self,prompt,history,model,pdf_path=None):
        pdf_text = None
        if pdf_path:
            pdf_text = self.pdf_extractor(pdf_path)

        #craft prompt
        user_prompt= prompt
        if pdf_text:
            user_prompt += f"""Here is the study meterial:

            {pdf_text}"""
        messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": user_prompt}]

        # call models
        stream = []
        if model == "gemini":
            stream= self.gemini.send_message_stream(user_prompt)
        elif model == "llma" or model == "qwen":
            stream = self.ollama.chat.completions.create(
                model= self.models[model],
                messages=messages,
                temperature = 0.7,
                stream= True
            )
        else:
            print("invalid model")
            return

        res = ""
        for chunk in stream:
            if model == "gemini":
                res += chunk.text or ""
            else:
                res += chunk.choices[0].delta.content or ''
            yield res
       

## And then enter Gradio's magic!

In [ ]:
assistant = StudyAssistant(gemini_api_key)
gr.ChatInterface(fn=assistant.chat, additional_inputs=[gr.Dropdown(["gemini", "qwen","llma"], label="Select model", value="gemini"),gr.File(label="upload pdf")], type="messages").launch()